# RewardFunction15
```
Key Elements:
1. New base function
2. Has base reward, reward progress relative to steps, reward speed based on curvature of track, and gives a small 
   reward for being left of center
3. Curvature calculations based on bezier curves
```

In [1]:
def reward_function(params):
    '''
    Base model where curvature is determined by bezier curves
    '''
   
    # Read input parameters
    #x_car = params['x']
    #y_car = params['y']
    on_track = params['all_wheels_on_track']
    distance_from_center = params['distance_from_center']
    track_width = params['track_width']
    progress = params['progress']/100
    waypoints = params['waypoints']
    closest_waypoints = params['closest_waypoints']
    steering = abs(params['steering_angle']) # Only need the absolute steering angle
    speed = params['speed']
    steps = params['steps']
    is_left_of_center = params['is_left_of_center']
    
    REWARD_MIN = 0.1
    REWARD_MAX = 100
    TARGET_STEPS = 300
    
    trackStraight = True
    
    # Calculate 3 marks that are farther and father away from the center line
    marker_1 = 0.25 * track_width
    marker_2 = 0.4 * track_width
    marker_3 = 0.6 * track_width
    
    # Set Base Reward
    if not on_track: # Fail them if off Track
        return REWARD_MIN
    elif progress == 1:
        #the lap is complete. if we use more steps than TARGET_STEPS, the reward is lower
        return REWARD_MAX * TARGET_STEPS / steps
    else:        
        reward = REWARD_MAX * max(progress, 0.4)
    
    # Determine waypoints
    prev_pt = waypoints[closest_waypoints[0]] # closest waypoint behind car
    next_pt = waypoints[closest_waypoints[1]] # closest waypoint ahead of car
    nextnext_pt = waypoints[(closest_waypoints[1] + 1) % len(waypoints)] # waypoint after next_pt
    
    prev_next_dist = ((prev_pt[0] - next_pt[0])**2 + (prev_pt[1] - next_pt[1])**2)**0.5    
    next_nextnext_dist = ((next_pt[0] - nextnext_pt[0])**2 + (next_pt[1] - nextnext_pt[1])**2)**0.5   
    prev_nextnext_dist = ((prev_pt[0] - nextnext_pt[0])**2 + (prev_pt[1] - nextnext_pt[1])**2)**0.5

    # Find out whether track is straight or curved
    curveCalculation = True
    myNumerator = (next_pt[0] - prev_pt[0])*(prev_pt[0] - 2 * next_pt[0] + nextnext_pt[0]) + (next_pt[1] - prev_pt[1])*(prev_pt[1] - 2 * next_pt[1] + nextnext_pt[1])
    myDenominator = (prev_pt[0] - 2 * next_pt[0] + nextnext_pt[0])**2 + (prev_pt[1] - 2 * next_pt[1] + nextnext_pt[1])**2
    
    #assuming that myDenominator does not equal zero
    if (myDenominator != 0):
        curvatureTime = myNumerator / myDenominator
        x_prime = 2 * prev_pt[0] * (curvatureTime - 1) + 2 * next_pt[0] - 4 * next_pt[0] * curvatureTime + 2 * nextnext_pt[0] * curvatureTime
        x_double_prime = 2 * prev_pt[0] - 4 * next_pt[0] + 2 * nextnext_pt[0]
        y_prime = 2 * prev_pt[1] * (curvatureTime - 1) + 2 * next_pt[1] - 4 * next_pt[1] * curvatureTime + 2 * nextnext_pt[1] * curvatureTime
        y_double_prime = 2 * prev_pt[1] - 4 * next_pt[1] + 2 * nextnext_pt[1]   
        temp_value = (x_prime**2 + y_prime**2)**1.5
    
        #assuming that temp_value does not equal zero
        if temp_value != 0:
            curvature = abs(x_prime * y_double_prime - x_double_prime * y_prime) / temp_value
        else:
            curveCalculation = False
    else:
        curveCalculation = False
    # Find out whether track is straight or curved
    if curveCalculation:
        if curvature < .0001:
            trackStraight = True
        else:
            trackStraight = False
    else:
        trackStraight = True
        
    # Reward based on whether track is straight or curved
    if trackStraight:
        #track is straight
        reward += (speed**3)/3.75 + (progress*900/(((steps**2) + 1)**.5))
        if is_left_of_center:
            reward += 1
    else:
        #track is curved
        if distance_from_center <= marker_2:
            reward += 5
        elif distance_from_center <= marker_3:
            reward += 3
        reward += ((500*progress)/(((steps**2) + 1)**.5))
    
    if is_left_of_center:
        reward += 1
    
    return float(reward)
